## Выбор параметров моделей и кросс-валидация

**Основная задача** обучающых алгоритмов - способность обощаться, то есть хорошо работать на новых данных. 
Чтобы оценить качество модели, нужно пожертвовать некоторой частью данных, чтобы проверить на ней качетсво модели.

***Есть 2 подхода:***
- **Разделения на тренировочную\тестовую выборки.** (held-out/hold-out set). Выделяем 20-40% тренировочной выборки и потом на ней оцениваем качество можедели. Считаем некоторую метрику. Саммое простое - долю правильных ответов в задаче классификации на отложенной выборке. 
- **Кросс-валидация** (cross-validation). Самый частый случай *k-fold cross-validation*.
![alt text](https://mlcourse.ai/_images/cross_validation.png) 

При k-fold кросс-валидации, модель тренируется $K$ раз на разных $K-1$ подвыбоках исходной выборки, а проверяется на оставшейся. Получаются $K$ оценок модели, которын обычео усредняются.

Кросс-валидация - очень важная техника в МО, с ее помошью выбираются гиперпараметры моделей, сравниваются модели, оценивантся полезность новых признаков.

## Пример
Деревья решения и knn в задачи предсказания оттока пользователей

In [5]:
import pandas as pd
import numpy as np
DATA_PATH = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/main/data/"

df = pd.read_csv(DATA_PATH + "telecom_churn.csv")
df["International plan"] = pd.factorize(df["International plan"])[0]
df["Voice mail plan"] = pd.factorize(df["Voice mail plan"])[0]
df["Churn"] = df["Churn"].astype("int")
states = df["State"]
y = df["Churn"]
df.drop(["State", "Churn"], axis=1, inplace=True)

df.head()


,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls
0,128,415,0,0,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,107,415,0,0,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,137,415,0,1,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,84,408,1,1,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,75,415,1,1,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3


Поделим выборку на 70% и 30%. параметры возьмем наугад

In [6]:
from sklearn.model_selection import StratifiedKFold, train_test_split 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

X_train, X_holdout, y_train, y_holdout = train_test_split(
    df.values, y, test_size=.3, random_state=17
)

tree = DecisionTreeClassifier(max_depth=5, random_state=17)
knn = KNeighborsClassifier(n_neighbors=10)

tree.fit(X_train, y_train)

# для knn нужно масштабировать фичи z = (x - u) / s
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.fit_transform(X_holdout)
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=10)

Качество прогноза проверим, с помошью простой метрики - доли правильных ответов. 

In [7]:
from sklearn.metrics import accuracy_score

tree_pred = tree.predict(X_holdout)
accuracy_score(y_holdout, tree_pred)

0.94

In [8]:
knn_pred = knn.predict(X_holdout_scaled)
accuracy_score(y_holdout, knn_pred)

0.886

Теперь подберем параметры дерева с помощью кросс-валидации. Будем искать макс глубину и максимальное используемое на каждом разбиении число признаков.

`GridSearchCV`: для каждой уникальной пары параметров max_depth и max_features будет проведена 5-кратная кросс-валидация и выберестся лучшее сочетание параметров.

In [9]:
from sklearn.model_selection import GridSearchCV, cross_val_score
tree_params = {"max_depth": range(1, 11), "max_features": range(4, 19)}
tree_grid = GridSearchCV(tree, tree_params, cv=5, n_jobs=-1, verbose=True)
tree_grid.fit(X_train, y_train)

Fitting 5 folds for each of 150 candidates, totalling 750 fits


GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=5, random_state=17),
             n_jobs=-1,
             param_grid={'max_depth': range(1, 11),
                         'max_features': range(4, 19)},
             verbose=True)

In [11]:
tree_grid.best_params_, tree_grid.best_score_

({'max_depth': 6, 'max_features': 17}, np.float64(0.9421418790379649))

In [13]:
accuracy_score(y_holdout, tree_grid.predict(X_holdout))

0.946

**Теперь настроим для knn**

In [14]:
from sklearn.pipeline import Pipeline

knn_pipe = Pipeline(
    [("scaler", StandardScaler()), ("knn", KNeighborsClassifier(n_jobs=-1))]
)

knn_params = {"knn__n_neighbors": range(1, 10)}
knn_grid = GridSearchCV(knn_pipe, knn_params, cv=5, n_jobs=-1, verbose=True)
knn_grid.fit(X_train, y_train)
knn_grid.best_params_, knn_grid.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


({'knn__n_neighbors': 7}, np.float64(0.8859867109023905))

In [15]:
accuracy_score(y_holdout, knn_grid.predict(X_holdout))

0.89

Здесь дерево решений, показало себя лучше, чем knn.